# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(file_path):
    file_paths = []
    for root, _, files in os.walk(file_path):
        file_paths.extend([os.path.join(root, file) for file in files if file.endswith(".json")])
    
    return file_paths

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data/')

In [5]:
file_path = song_files[0]
print(file_path)

data/song_data/A/B/B/TRABBVJ128F92F7EAA.json


In [6]:
df = pd.read_json(file_path, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AREDL271187FB40F44,NaN,NaN,,Soul Mekanik,SOPEGZN12AB0181B3D,Get Your Head Stuck On Your Neck,45.66159,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
(_, artist_id, artist_latitude, artist_longitude, artist_location,
         artist_name, song_id, title, duration, year) = df.values[0]

In [8]:
# (song_id text, title text, artist_id int, year int, duration numeric)
song_data = [song_id, title, artist_id, year, duration]
song_data

['SOPEGZN12AB0181B3D',
 'Get Your Head Stuck On Your Neck',
 'AREDL271187FB40F44',
 0,
 45.66159]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(songs_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
#  (artist_id text, name text, location text, latitude float, longitude float)
artist_data = [artist_id, artist_name, artist_location, artist_longitude, artist_latitude]
artist_data

['AREDL271187FB40F44', 'Soul Mekanik', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artists_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files('data/log_data/')

In [14]:
file_path = log_files[0]
print(file_path)

data/log_data/2018/11/2018-11-05-events.json


In [15]:
df = pd.read_json(file_path, lines=True)
print(df.size)
df.head()

7344


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,256,Almost Lover (Album Version),200,1541377992796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Serve The Servants,200,1541381242796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,Television,Logged In,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,See No Evil (Remastered LP Version),200,1541381456796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,JOHN COLTRANE,Logged In,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Blues To Bechet (LP Version),200,1541381694796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,NOFX,Logged In,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,It's My Job To Keep Punk Rock Elite,200,1541382040796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
df = df[df['page']=='NextSong']
print(df.size)
df.head()

6408


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,256,Almost Lover (Album Version),200,1541377992796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Nirvana,Logged In,Aleena,F,0,Kirby,214.77832,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Serve The Servants,200,1541381242796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
2,Television,Logged In,Aleena,F,1,Kirby,238.49751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,See No Evil (Remastered LP Version),200,1541381456796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,JOHN COLTRANE,Logged In,Aleena,F,2,Kirby,346.43546,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,Blues To Bechet (LP Version),200,1541381694796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
4,NOFX,Logged In,Aleena,F,3,Kirby,80.79628,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1.541023e+12,237,It's My Job To Keep Punk Rock Elite,200,1541382040796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44


In [18]:
t = pd.to_datetime(df['ts'], unit='ms') 
t.head()

0   2018-11-05 00:33:12.796
1   2018-11-05 01:27:22.796
2   2018-11-05 01:30:56.796
3   2018-11-05 01:34:54.796
4   2018-11-05 01:40:40.796
Name: ts, dtype: datetime64[ns]

In [19]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set time_data to a list containing these values in order
time_data = [[line, line.hour, line.day, line.week, line.month, line.year, line.day_name()] for line in t]
for row in time_data:
    cur.execute(time_insert, row)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [20]:
time_data[1]

[Timestamp('2018-11-05 01:27:22.796000'), 1, 5, 45, 11, 2018, 'Monday']

In [21]:
# Create a dataframe, time_df, containing the time data for this file by combining column_labels and time_data into a dictionary and converting this into a dataframe
time_df = pd.DataFrame.from_records(time_data, columns=column_labels)
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-05 00:33:12.796,0,5,45,11,2018,Monday
1,2018-11-05 01:27:22.796,1,5,45,11,2018,Monday
2,2018-11-05 01:30:56.796,1,5,45,11,2018,Monday
3,2018-11-05 01:34:54.796,1,5,45,11,2018,Monday
4,2018-11-05 01:40:40.796,1,5,45,11,2018,Monday


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for _, row in time_df.iterrows():
    cur.execute(time_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
1,44,Aleena,Kirby,F,paid
2,44,Aleena,Kirby,F,paid
3,44,Aleena,Kirby,F,paid
4,44,Aleena,Kirby,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(users_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
# insert songplay records
for _, row in df.iterrows():
    # get song_id and artist_id from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print(results) if results else None
    song_id, artist_id = results if results else (None, None)
    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), int(
        row.userId), row.level, song_id, artist_id, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_insert, songplay_data)
conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.